In [1]:
import pandas as pd
import gc

In [36]:
# sunat_train = pd.read_csv('../data/sunat_train.csv')
rcc_train = pd.read_csv('../data/rcc_train.csv')
se_train = pd.read_csv('../data/se_train.csv', index_col = 'key_value')
censo_train = pd.read_csv('../data/censo_train.csv', index_col = 'key_value')
y_train = pd.read_csv('../data/y_train.csv', index_col = 'key_value')

# sunat_test = pd.read_csv('../data/sunat_test.csv')
rcc_test = pd.read_csv('../data/rcc_test.csv')
se_test = pd.read_csv('../data/se_test.csv', index_col = 'key_value')
censo_test = pd.read_csv('../data/censo_test.csv', index_col = 'key_value')
sample_submision = pd.read_csv('../data/sample_submission.csv', index_col = 'key_value')

In [3]:
### se puede observar que en el rcc_train aquellos clientes que tienen un solo registro, es porque se refiere al ultimo
### periodo de la data de train que es del febrero del 2018, cuyo RIESGO_DIRECTO es 1 y COD_CLASIFICACION_DEUDOR es 0 
### su condicion (días de atraso) es 0 siendo estos cerca de 623 clientes. Pero OJO el 9% de los nuevos clientes tienen un target igual a 1

### el target de la data train representa el score crediticio al febrero del 2018 
### por lo que se sobre-entiende que lo que se quiere predecir es el score crediticio al febrero del 2019

### La data train abarca desde 201703 hasta 201802
### La data test comprende desde 201803 hasta 201902
### Los primeros registros de los clientes tienen un RIESGO DIRECTO de 1 ***

### La distribucion del target general es de 16% para 1 y 84% para los 0
### A mayor COD_CLASIFICACION_DEUDOR y codmes mes cercano, el target se aproxima a 1. Ejm COD_CLASIFICACION_DEUDOR=5 y codmes=201802
### el target esta representado por el 97% de 1 y 3% de cero 

In [ ]:
### condicion, saldo   ..... son variables cuantitativas
### tipo_credito en train no tiene la clase de 7 como en el test, pero este tiene solo 1 valor (8 clases) ### Frecuencia 11
### cod_instit_financiera , 12 clases más se encuentran en el train que en el test (mas de 100 clases) ### Frecuencia 34 ----
### PRODUCTO, el train tiene la clase 255 y el test no (42 clases) ### Frecuencia 1  ----
### RIESGO_DIRECTO es lo mismo , es lo mismo en el test y en el train (5 clases) ### Frecuencia -1
### COD_CLASIFICACION_DEUDOR , es lo mismo en el test y en el train (6 clases) ### Frecuencia 0
### en el rcc_test, faltan valores en la columma de cod_instit_financiera y producto

In [ ]:
# rcc_train_class_deudor =rcc_train.groupby(['key_value','codmes', "COD_CLASIFICACION_DEUDOR"]).saldo.sum().unstack(level = 2, fill_value=0).reset_index().set_index("key_value")
# rcc_train_class_deudor.columns = ["class_deudor_" + str(c) if c != "codmes" else c for c in rcc_train_class_deudor.columns ]
# rcc_train_class_deudor

In [3]:
import statistics as stat
import numpy as np
def calculate_mode(x):
    try:
        moda=stat.mode(x)
    except:
        moda=np.nan
    return moda

In [4]:
from tqdm.notebook import tqdm 
moda=lambda x: calculate_mode(x)
moda.__name__='mode'
agg_rcc = {'saldo':['sum','std'],
           'condicion':['sum','max','std'],
           'tipo_credito':['nunique', moda],
           'cod_instit_financiera':['nunique',moda],
           'PRODUCTO':['nunique',moda],
           'RIESGO_DIRECTO':['nunique',moda],
            'COD_CLASIFICACION_DEUDOR':['nunique','max',moda]
          }
list_rcc_train_agg = []
for n,i in enumerate(tqdm(sorted(set(rcc_train.codmes),reverse=True))):
    rcc_train_agg = rcc_train[rcc_train.codmes>=i].groupby('key_value').agg(agg_rcc)
    rcc_train_agg.columns = [i+'_'+j+f'_ult{n+1}mes' for i,j in rcc_train_agg.columns]
    list_rcc_train_agg.append(rcc_train_agg)
del rcc_train, list_rcc_train_agg

In [5]:
list_rcc_test_agg = []
for n,i in enumerate(tqdm(sorted(set(rcc_test.codmes),reverse=True))):
    rcc_test_agg = rcc_test[rcc_test.codmes>=i].groupby('key_value').agg(agg_rcc)
    rcc_test_agg.columns = [i+'_'+j+f'_ult{n+1}mes' for i,j in rcc_test_agg.columns]
    list_rcc_test_agg.append(rcc_test_agg)
del rcc_test, list_rcc_test_agg

In [37]:
train = rcc_train_.join(censo_train).join(se_train)
test = rcc_test_.join(censo_test).join(se_test)

In [ ]:
# rcc_test_ = pd.concat(list_rcc_test_agg, axis=1)
# for i in rcc_test_.columns:
#     if 'RIESGO_DIRECTO_unique' in i:
#         print(i)
#         rcc_test_[i] = rcc_test_[i].apply(lambda x: len(x))
# rcc_test_.rename(columns = dict_, inplace = True)

In [39]:
keep_cols = list(set(train.columns).intersection(set(test.columns)))
train = train[keep_cols]
test = test[keep_cols]
len(set(train.columns) - set(test.columns)) , len(set(test.columns) - set(train.columns))

(0, 0)

In [42]:
import re
test = test.rename(columns = lambda x:re.sub('[^A-Za-z0-9_-]+', '', x))
train = train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_-]+', '', x))

In [43]:
#Función para obtener variables con alta correlación
def get_correlated(data, cut=0.85):
    correlated_features = set()
    correlation_matrix = data.corr()
    
    for i in range(len(correlation_matrix .columns)):
        for j in range(i):
            if abs(correlation_matrix.iloc[i, j]) > cut:
                colname = correlation_matrix.columns[i]
                correlated_features.add(colname)
            
    return correlated_features 

In [44]:
train.shape

(358487, 461)

In [45]:
correlated=get_correlated(train,cut=0.9)
train=train.drop(columns=correlated)
test= test.drop(columns=correlated)

In [48]:
train.isna().sum()

tipo_credito_mode_ult7mes                   3598
cod_instit_financiera_nunique_ult12mes         0
condicion_max_ult12mes                         0
mattec2                                   205348
nhogares                                  205348
                                           ...  
lanten                                    205348
ctd_veh                                       59
pred                                      205348
matpar8                                   205348
lee_no                                    205348
Length: 162, dtype: int64

In [49]:
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold

folds = [train.index[t] for t, v in KFold(5).split(train)]
test_probs = []
train_probs = []
fi = []
for i, idx in enumerate(folds):
    print("*"*10, i, "*"*10)
    Xt = train.loc[idx]
    yt = y_train.loc[Xt.index]

    Xv = train.drop(Xt.index)
    yv = y_train.loc[Xv.index]
    
    learner = LGBMClassifier(n_estimators=1000)
    learner.fit(Xt, yt,  early_stopping_rounds=10, eval_metric="auc",
                eval_set=[(Xt, yt), (Xv, yv)], verbose=50)
    test_probs.append(pd.Series(learner.predict_proba(test)[:, -1], index=test.index, name="fold_" + str(i)))
    train_probs.append(pd.Series(learner.predict_proba(Xv)[:, -1], index=Xv.index, name="probs"))
    fi.append(pd.Series(learner.feature_importances_ / learner.feature_importances_.sum(), index=Xt.columns))
          
test_probs = pd.concat(test_probs, axis=1).mean(axis=1)
train_probs = pd.concat(train_probs)
fi = pd.concat(fi, axis=1).mean(axis=1)

print("*" * 21)
print("roc auc estimado: ", roc_auc_score(y_train, train_probs.loc[y_train.index]))

********** 0 **********


/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.845178	training's binary_logloss: 0.300248	valid_1's auc: 0.842685	valid_1's binary_logloss: 0.304199
[100]	training's auc: 0.858044	training's binary_logloss: 0.291546	valid_1's auc: 0.847195	valid_1's binary_logloss: 0.30023
Early stopping, best iteration is:
[105]	training's auc: 0.859069	training's binary_logloss: 0.2909	valid_1's auc: 0.847376	valid_1's binary_logloss: 0.300089
********** 1 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.846145	training's binary_logloss: 0.300028	valid_1's auc: 0.839773	valid_1's binary_logloss: 0.3045
[100]	training's auc: 0.858301	training's binary_logloss: 0.291538	valid_1's auc: 0.844217	valid_1's binary_logloss: 0.300659
[150]	training's auc: 0.867475	training's binary_logloss: 0.285932	valid_1's auc: 0.844896	valid_1's binary_logloss: 0.299992
Early stopping, best iteration is:
[189]	training's auc: 0.873351	trai

In [50]:
fi.sort_values(ascending=False).to_frame()

,0
cod_instit_financiera_mode_ult6mes,0.067213
saldo_sum_ult11mes,0.059623
saldo_std_ult1mes,0.047170
cod_instit_financiera_nunique_ult12mes,0.045086
edad,0.043799
...,...
pagagu3,0.000000
mattec8,0.000000
matpis7,0.000000
pagagu2,0.000000


In [51]:
test_probs.name = "target"
test_probs.to_csv("test1.csv")

### Probando con CatBoost

In [59]:
# !pip install catboost
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold

folds = [train.index[t] for t, v in KFold(5).split(train)]
test_probs = []
train_probs = []
fi = []
for i, idx in enumerate(folds):
    print("*"*10, i, "*"*10)
    Xt = train.loc[idx]
    yt = y_train.loc[Xt.index]

    Xv = train.drop(Xt.index)
    yv = y_train.loc[Xv.index]
    
    learner = CatBoostClassifier(n_estimators=1000, depth = 6, eval_metric = 'AUC' )
    learner.fit(Xt, yt,  early_stopping_rounds=10,
                eval_set=[(Xt, yt), (Xv, yv)], verbose=50)
    test_probs.append(pd.Series(learner.predict_proba(test)[:, -1], index=test.index, name="fold_" + str(i)))
    train_probs.append(pd.Series(learner.predict_proba(Xv)[:, -1], index=Xv.index, name="probs"))
    fi.append(pd.Series(learner.feature_importances_ / learner.feature_importances_.sum(), index=Xt.columns))
          
test_probs = pd.concat(test_probs, axis=1).mean(axis=1)
train_probs = pd.concat(train_probs)
fi = pd.concat(fi, axis=1).mean(axis=1)

print("*" * 21)
print("roc auc estimado: ", roc_auc_score(y_train, train_probs.loc[y_train.index]))

********** 0 **********
Learning rate set to 0.128212
0:	test: 0.7550204	test1: 0.7637180	best: 0.7637180 (0)	total: 134ms	remaining: 2m 14s
50:	test: 0.8348013	test1: 0.8365475	best: 0.8365475 (50)	total: 8.51s	remaining: 2m 38s
100:	test: 0.8429696	test1: 0.8427612	best: 0.8427612 (100)	total: 16.7s	remaining: 2m 28s
150:	test: 0.8478433	test1: 0.8458935	best: 0.8458935 (150)	total: 24.5s	remaining: 2m 17s
200:	test: 0.8511124	test1: 0.8472869	best: 0.8472869 (200)	total: 31.9s	remaining: 2m 6s
250:	test: 0.8539636	test1: 0.8482192	best: 0.8482192 (250)	total: 39.4s	remaining: 1m 57s
300:	test: 0.8562315	test1: 0.8488041	best: 0.8488041 (300)	total: 46.8s	remaining: 1m 48s
Stopped by overfitting detector  (10 iterations wait)

bestTest = 0.8492821952
bestIteration = 334

Shrink model to first 335 iterations.
********** 1 **********
Learning rate set to 0.128212
0:	test: 0.7572985	test1: 0.7561405	best: 0.7561405 (0)	total: 131ms	remaining: 2m 10s
50:	test: 0.8360436	test1: 0.8327414	

In [72]:
fi.sort_values(ascending=False).head(30)#.plot.bar()

COD_CLASIFICACION_DEUDOR_max_ult1mes        0.094128
cod_instit_financiera_nunique_ult12mes      0.061666
cod_instit_financiera_mode_ult6mes          0.059939
saldo_sum_ult11mes                          0.053627
PRODUCTO_mode_ult1mes                       0.038461
condicion_sum_ult2mes                       0.037019
saldo_std_ult1mes                           0.036627
COD_CLASIFICACION_DEUDOR_nunique_ult1mes    0.029923
edad                                        0.026519
profe_1                                     0.026121
tipo_credito_mode_ult1mes                   0.024416
saldo_std_ult8mes                           0.023365
matpar7                                     0.022033
PRODUCTO_nunique_ult12mes                   0.021203
condicion_max_ult12mes                      0.019920
tvteno                                      0.019874
tip_lvledu                                  0.019860
cod_ubi                                     0.016282
tipo_credito_mode_ult11mes                  0.

In [61]:
test_probs.name = "target"
test_probs.to_csv("test3.csv")